## Chapter7.8 DNNClassifierで簡単学習

In [1]:
# パッケージの読み込み
import tensorflow as tf
import numpy as np
import pandas as pd

データセットの読み込み
<font color="red">**PROJECTID**を書き換えて実行してください</font>

In [2]:
# BigQueryクエリ結果をDataFrameに読み込む
query = 'SELECT * FROM testdataset.wdbc ORDER BY index'
dataset = pd.read_gbq(project_id='PROJECTID', query=query)

# データの先頭５行を表示
dataset.head()

Requesting query... ok.
Query running...
Query done.
Cache hit.

Retrieving results...
Got 569 rows.

Total time taken 1.73 s.
Finished at 2017-09-26 23:08:14.


,index,diagnostic,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,mean_compactness,mean_concavity,mean_concave_points,...,worst_radius,worst_texture,worst_perimeter,worst_area,worst_smoothness,worst_compactness,worst_concavity,worst_concave_points,worst_symmetry,worst_fractal_dimension
0,8670,M,15.46,19.48,101.70,748.9,0.10920,0.12230,0.14660,0.08087,...,19.26,26.00,124.90,1156.0,0.15460,0.2394,0.3791,0.15140,0.2837,0.08019
1,8913,B,12.89,13.12,81.89,515.9,0.06955,0.03729,0.02260,0.01171,...,13.62,15.54,87.40,577.0,0.09616,0.1147,0.1186,0.05366,0.2309,0.06915
2,8915,B,14.96,19.10,97.03,687.3,0.08992,0.09823,0.05940,0.04819,...,16.25,26.19,109.10,809.8,0.13130,0.3030,0.1804,0.14890,0.2962,0.08472
3,9047,B,12.94,16.17,83.18,507.6,0.09879,0.08836,0.03296,0.02390,...,13.86,23.02,89.69,580.9,0.11720,0.1958,0.1810,0.08388,0.3297,0.07834
4,85715,M,13.17,18.66,85.98,534.6,0.11580,0.12310,0.12260,0.07340,...,15.67,27.95,102.80,759.4,0.17860,0.4166,0.5006,0.20880,0.3900,0.11790


データの整理

In [3]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# 'M'を0, 'B'を1に変換
dataset['diagnostic'] = dataset['diagnostic'].apply(
    lambda x: 0 if x == 'M' else 1)

# 'index'カラムを削除
dataset.drop('index', axis=1, inplace=True)

# DataFrameからarrayに変換
X_dataset = dataset.drop('diagnostic', axis=1).as_matrix()
y_dataset = dataset.diagnostic.as_matrix()

# 学習用とテスト用にデータセットを分ける
X_train, X_test, y_train, y_test = train_test_split(
    X_dataset, y_dataset, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

DNNClassifierで学習

In [4]:
from tensorflow.contrib.learn.python import SKCompat

# 特徴量の数を定義
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=30)]

# 隠れ層2つ、それぞれ4ユニット
classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns,
                                            hidden_units=[4, 4],
                                            n_classes=2,
                                            model_dir="./dnnmodel/")
# scikit-learnライクSKCompatに変換
classifier = SKCompat(classifier)

# 学習
classifier.fit(x=X_train, y=y_train, steps=1000, batch_size=50)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11bd494d0>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./dnnmodel/model.ckpt.


SKCompat()

テストデータで推論し正解率算出

In [5]:
classifier.score(X_test, y_test)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-09-26-14:08:23
INFO:tensorflow:Finished evaluation at 2017-09-26-14:08:24
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.95614, accuracy/baseline_label_mean = 0.596491, accuracy/threshold_0.500000_mean = 0.95614, auc = 0.981458, global_step = 1000, labels/actual_label_mean = 0.596491, labels/prediction_mean = 0.624474, loss = 0.20413, precision/positive_threshold_0.500000_mean = 0.970149, recall/positive_threshold_0.500000_mean = 0.955882


{'accuracy': 0.95614034,
 'accuracy/baseline_label_mean': 0.59649122,
 'accuracy/threshold_0.500000_mean': 0.95614034,
 'auc': 0.98145783,
 'global_step': 1000,
 'labels/actual_label_mean': 0.59649122,
 'labels/prediction_mean': 0.62447393,
 'loss': 0.20413011,
 'precision/positive_threshold_0.500000_mean': 0.97014928,
 'recall/positive_threshold_0.500000_mean': 0.95588237}